In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from scipy.stats import beta

In [3]:
import scipy.integrate as integrate
import scipy.special as special

In [4]:
class ExpoAlloc:

    def __init__(
        self,
        lbd,
        alpha,
        thetamax,
        ):
        self.lbd = lbd
        self.alpha = alpha
        self.thetamax = thetamax
        self.vstar = -np.log(1 - self.thetamax) / self.lbd

    def phi(self, x):
        return x - 1

    def alloc(self, arr):
        v2r = self.phi(arr[:, 1])
        v1r = self.phi(arr[:, 0]) / self.alpha
        cpr = np.maximum(v1r, self.phi(self.vstar))
        return (v2r <= cpr).astype(int)

    def F(self, x):
        return np.power(np.e, -self.lbd * x)

    def interimalloc(self, x):
        return -self.alpha + self.alpha * (1 - np.power(np.e, -self.lbd
                * (1 + self.alpha * (x - 1)))) * int(x > self.vstar) \
            + max(1 - np.power(np.e, -self.lbd * self.vstar), 1
                  - np.power(np.e, -self.lbd * (1 + (x - 1)
                  / self.alpha)))

    def scipyprice(self, x):
        return x * self.interimalloc(x) - integrate.quad(lambda x: \
                self.interimalloc(x), 0, x)[0]

    def error(self, x):
        return integrate.quad(lambda x: self.interimalloc(x), 0, x)[1]

    def sample(self, batch_size):
        return np.random.exponential(scale=self.lbd, size=batch_size)

    def price(self, x):
        breakpoint1 = self.vstar
        breakpoint2 = 1 + self.alpha * (self.vstar - 1)
        if breakpoint1 <= breakpoint2:
            if x <= breakpoint1:
                return 0
            elif x > breakpoint1 and x <= breakpoint2:
                fstpart = -((1 - self.alpha - np.power(np.e, -self.lbd
                            * self.vstar)) * breakpoint1)
                return self.interimalloc(x) * x + fstpart \
                    - np.power(np.e, self.lbd * (-1 + self.alpha
                               - self.alpha * x)) / self.lbd \
                    + np.power(np.e, self.lbd * (-1 + self.alpha
                               - self.alpha * breakpoint1)) / self.lbd \
                    - x + breakpoint1 - np.power(np.e, -self.lbd
                        * self.vstar) * (breakpoint1 - x)
            else:
                fstpart = -((1 - self.alpha - np.power(np.e, -self.lbd
                            * self.vstar)) * breakpoint1)
                sndpart = -np.power(np.e, self.lbd * (-1 + self.alpha
                                    - self.alpha * breakpoint2)) \
                    / self.lbd + np.power(np.e, self.lbd * (-1
                        + self.alpha - self.alpha * breakpoint1)) \
                    / self.lbd - breakpoint2 + breakpoint1 \
                    - np.power(np.e, -self.lbd * self.vstar) \
                    * (breakpoint1 - breakpoint2)
                return self.interimalloc(x) * x + fstpart + sndpart \
                    - np.power(np.e, self.lbd * (-1 + self.alpha
                               - self.alpha * x)) / self.lbd \
                    + np.power(np.e, self.lbd * (-1 + self.alpha
                               - self.alpha * breakpoint2)) / self.lbd \
                    - self.alpha * (np.power(np.e, -self.lbd * (-1
                                    + self.alpha + x) / self.alpha)
                                    - np.power(np.e, -self.lbd * (-1
                                    + self.alpha + breakpoint2)
                                    / self.alpha)) / self.lbd - x \
                    + breakpoint2
        else:
            if x <= breakpoint2:
                return 0
            elif x > breakpoint2 and x <= breakpoint1:
                fstpart = -((1 - self.alpha - np.power(np.e, -self.lbd
                            * self.vstar)) * breakpoint2)
                return self.interimalloc(x) * x + fstpart - x \
                    + breakpoint2 - self.alpha * (np.power(np.e,
                        -self.lbd * (-1 + self.alpha + x) / self.alpha)
                        - np.power(np.e, -self.lbd * (-1 + self.alpha
                        + breakpoint2) / self.alpha) + self.lbd
                        * (breakpoint2 - x)) / self.lbd
            else:
                fstpart = -((1 - self.alpha - np.power(np.e, -self.lbd
                            * self.vstar)) * breakpoint2)
                sndpart = -breakpoint1 + breakpoint2 - self.alpha \
                    * (np.power(np.e, -self.lbd * (-1 + self.alpha
                       + breakpoint1) / self.alpha) - np.power(np.e,
                       -self.lbd * (-1 + self.alpha + breakpoint2)
                       / self.alpha) + self.lbd * (breakpoint2
                       - breakpoint1)) / self.lbd
                return self.interimalloc(x) * x + fstpart + sndpart \
                    - np.power(np.e, self.lbd * (-1 + self.alpha
                               - self.alpha * x)) / self.lbd \
                    + np.power(np.e, self.lbd * (-1 + self.alpha
                               - self.alpha * breakpoint1)) / self.lbd \
                    - self.alpha * (np.power(np.e, -self.lbd * (-1
                                    + self.alpha + x) / self.alpha)
                                    - np.power(np.e, -self.lbd * (-1
                                    + self.alpha + breakpoint1)
                                    / self.alpha)) / self.lbd - x \
                    + breakpoint1

    def revenue(self, batchsize):
        x = self.sample(batchsize).reshape(-1, 2)
        y = x.copy()
        y[:, 0] = x[:, 1]
        y[:, 1] = x[:, 0]
        return ((self.phi(x[:, 0]) - self.alpha * self.phi(x[:, 1]))
                * self.alloc(x)).mean() + ((self.phi(x[:, 1])
                - self.alpha * self.phi(x[:, 0]))
                * self.alloc(y)).mean()

    def revenue2(self, batchsize):
        x = self.sample(batchsize)
        ctr = 0
        count = 0
        for i in x:
            ctr += self.interimalloc(i) * self.phi(i) * 2
            count += 1
        return ctr / count


In [5]:
class Uniform1alloc:

    def __init__(self, alpha, thetamax):
        self.alpha = alpha
        self.thetamax = thetamax
        self.vstar = thetamax

    def phi(self, x):
        return 2 * x - 1

    def iphi(self, x):
        return x / 2 + 1 / 2

    def alloc(self, arr):
        v2r = self.phi(arr[:, 1])
        v1r = self.phi(arr[:, 0]) / self.alpha
        cpr = np.maximum(v1r, self.phi(self.vstar))
        return (v2r <= cpr).astype(int)

    def F(self, x):
        if x < 0.0:
            return 0
        elif x < 1.0:
            return x
        elif x >= 1.0:
            return 1
        else:
            print (x, 'unexpected CDF value')

    def interimalloc(self, x):
        return max(self.F(self.vstar), self.F(self.iphi(self.phi(x)
                   / self.alpha))) + self.alpha * int(x > self.vstar) \
            * self.F(self.iphi(self.alpha * self.phi(x))) - self.alpha

    def scipyprice(self, x):
        return x * self.interimalloc(x) - integrate.quad(lambda x: \
                self.interimalloc(x), 0, x)[0]

    def error(self, x):
        return integrate.quad(lambda x: self.interimalloc(x), 0, x)[1]

    def sample(self, batch_size):
        return np.random.uniform(low=0.0, high=1.0, size=batch_size)

    def price(self, x):
        breakpoint1 = self.vstar
        breakpoint2 = 1 / 2 + 1 / 2 * self.alpha * (2 * self.vstar - 1)
        if breakpoint1 <= breakpoint2:
            if x <= breakpoint1:
                return 0
            elif x > breakpoint1 and x <= breakpoint2:
                fstpart = -((-self.alpha + self.vstar) * breakpoint1)
                return self.interimalloc(x) * x + fstpart + self.alpha \
                    * x / 2 + np.power(self.alpha, 2) * x / 2 \
                    - self.vstar * x - np.power(self.alpha, 2) \
                    * np.power(x, 2) / 2 - self.alpha * breakpoint1 / 2 \
                    - np.power(self.alpha, 2) * breakpoint1 / 2 \
                    + self.vstar * breakpoint1 + np.power(self.alpha,
                        2) * np.power(breakpoint1, 2) / 2
            else:
                fstpart = -((-self.alpha + self.vstar) * breakpoint1)
                sndpart = self.alpha * breakpoint2 / 2 \
                    + np.power(self.alpha, 2) * breakpoint2 / 2 \
                    - self.vstar * breakpoint2 - np.power(self.alpha,
                        2) * np.power(breakpoint2, 2) / 2 - self.alpha \
                    * breakpoint1 / 2 - np.power(self.alpha, 2) \
                    * breakpoint1 / 2 + self.vstar * breakpoint1 \
                    + np.power(self.alpha, 2) * np.power(breakpoint1,
                        2) / 2
                return self.interimalloc(x) * x + fstpart + sndpart - x \
                    / 2 + x / (2 * self.alpha) + self.alpha * x / 2 \
                    + np.power(self.alpha, 2) * x / 2 - np.power(x, 2) \
                    / (2 * self.alpha) - np.power(self.alpha, 2) \
                    * np.power(x, 2) / 2 + breakpoint2 / 2 \
                    - breakpoint2 / (2 * self.alpha) - self.alpha \
                    * breakpoint2 / 2 - np.power(self.alpha, 2) \
                    * breakpoint2 / 2 + np.power(breakpoint2, 2) / (2
                        * self.alpha) + np.power(self.alpha, 2) \
                    * np.power(breakpoint2, 2) / 2
        else:
            if x <= breakpoint2:
                return 0
            elif x > breakpoint2 and x <= breakpoint1:
                fstpart = -((-self.alpha + self.vstar) * breakpoint2)
                return self.interimalloc(x) * x + fstpart - x / 2 + x \
                    / (2 * self.alpha) + self.alpha * x - np.power(x,
                        2) / (2 * self.alpha) + breakpoint2 / 2 \
                    - breakpoint2 / (2 * self.alpha) - self.alpha \
                    * breakpoint2 + np.power(breakpoint2, 2) / (2
                        * self.alpha)
            else:
                fstpart = -((-self.alpha + self.vstar) * breakpoint2)
                sndpart = -breakpoint1 / 2 + breakpoint1 / (2
                        * self.alpha) + self.alpha * breakpoint1 \
                    - np.power(breakpoint1, 2) / (2 * self.alpha) \
                    + breakpoint2 / 2 - breakpoint2 / (2 * self.alpha) \
                    - self.alpha * breakpoint2 + np.power(breakpoint2,
                        2) / (2 * self.alpha)
                return self.interimalloc(x) * x + fstpart + sndpart - x \
                    / 2 + x / (2 * self.alpha) + self.alpha * x / 2 \
                    + np.power(self.alpha, 2) * x / 2 - np.power(x, 2) \
                    / (2 * self.alpha) - np.power(self.alpha, 2) \
                    * np.power(x, 2) / 2 + breakpoint1 / 2 \
                    - breakpoint1 / (2 * self.alpha) - self.alpha \
                    * breakpoint1 / 2 - np.power(self.alpha, 2) \
                    * breakpoint1 / 2 + np.power(breakpoint1, 2) / (2
                        * self.alpha) + np.power(self.alpha, 2) \
                    * np.power(breakpoint1, 2) / 2

    def revenue(self, batchsize):
        x = self.sample(batchsize).reshape(-1, 2)
        y = x.copy()
        y[:, 0] = x[:, 1]
        y[:, 1] = x[:, 0]
        return ((self.phi(x[:, 0]) - self.alpha * self.phi(x[:, 1]))
                * self.alloc(x)).mean() + ((self.phi(x[:, 1])
                - self.alpha * self.phi(x[:, 0]))
                * self.alloc(y)).mean()

    def revenue2(self, batchsize):
        x = self.sample(batchsize)
        ctr = 0
        count = 0
        for i in x:
            ctr += self.interimalloc(i) * self.phi(i) * 2
            count += 1
        return ctr / count


In [6]:
class AsymUniform:

    def __init__(self, alpha, thetamax):
        self.alpha = alpha
        self.thetamax = thetamax
        self.vstar1 = thetamax
        self.vstar2 = 2 * thetamax

    def phi1(self, x):
        return 2 * x - 1

    def phi2(self, x):
        return 2 * x - 2

    def iphi1(self, x):
        return x / 2 + 1 / 2

    def iphi2(self, x):
        return x / 2 + 1

    def alloc1(self, arr):
        v2r = self.phi2(arr[:, 1])
        v1r = self.phi1(arr[:, 0]) / self.alpha
        cpr = np.maximum(v1r, self.phi2(self.vstar2))
        return (v2r <= cpr).astype(int)

    def alloc2(self, arr):
        v1r = self.phi1(arr[:, 0])
        v2r = self.phi2(arr[:, 1]) / self.alpha
        cpr = np.maximum(v2r, self.phi1(self.vstar1))
        return (v1r <= cpr).astype(int)

    def F1(self, x):
        if x < 0:
            return 0
        elif x < 1:
            return x
        else:
            return 1

    def F2(self, x):
        if x < 0:
            return 0
        elif x < 2:
            return x / 2
        else:
            return 1

    def interimalloc1(self, x):
        return max(self.F2(self.vstar1),
                   self.F2(self.iphi1(self.phi1(x) / self.alpha))) \
            + self.alpha * int(x > self.vstar2) \
            * self.F1(self.iphi1(self.alpha * self.phi1(x))) \
            - self.alpha

    def interimalloc2(self, x):
        return max(self.F1(self.vstar2),
                   self.F1(self.iphi2(self.phi2(x) / self.alpha))) \
            + self.alpha * int(x > self.vstar1) \
            * self.F2(self.iphi2(self.alpha * self.phi2(x))) \
            - self.alpha

    def scipyprice1(self, x):
        return x * self.interimalloc1(x) - integrate.quad(lambda x: \
                self.interimalloc(x), 0, x)[0]

    def scipyprice2(self, x):
        return x * self.interimalloc2(x) - integrate.quad(lambda x: \
                self.interimalloc(x), 0, x)[0]

    def error(self, x):
        return integrate.quad(lambda x: self.interimalloc1(x), 0, x)[1]

    def sample(self, idx, batchsize):
        return np.random.uniform(low=0.0, high=idx + 1.0,
                                 size=batchsize)

    def revenue(self, batchsize):
        x = self.sample(0, batchsize)
        y = self.sample(1, batchsize)
        x = np.stack((x, y), -1)
        return ((self.phi1(x[:, 0]) - self.alpha * self.phi2(x[:, 1]))
                * self.alloc1(x)).mean() + ((self.phi2(x[:, 1])
                - self.alpha * self.phi1(x[:, 0]))
                * self.alloc2(x)).mean()

    def revenue2(self, batchsize):
        x = self.sample(0, batchsize)
        y = self.sample(1, batchsize)
        ctr = 0
        count = 0
        for ind in range(len(x)):
            i = x[ind]
            j = y[ind]
            ctr += self.interimalloc1(i) * self.phi1(i) \
                + self.interimalloc2(j) * self.phi2(j)
            count += 1
        return ctr / count


In [7]:
class Irregular:

    def __init__(self, alpha, thetamax):
        self.alpha = alpha
        self.thetamax = thetamax

    def phi(self, x):
        mask1 = x <= (7.0 - np.sqrt(5.0)) / 2.0
        mask2 = np.logical_and(x > (7.0 - np.sqrt(5.0)) / 2.0, x
                               <= (11.0 - np.sqrt(5.0)) / 2.0)
        mask3 = np.logical_and(x > (7.0 - np.sqrt(5.0)) / 2.0, x
                               > (11.0 - np.sqrt(5.0)) / 2.0)
        output = x.copy()
        output[mask1] = 2.0 * x[mask1] - 4.0
        output[mask2] = 3.0 - np.sqrt(5.0)
        output[mask3] = 2.0 * x[mask3] - 8.0
        return output

    def scalarphi(self, x):
        if x <= (7.0 - np.sqrt(5.0)) / 2.0:
            return 2.0 * x - 4.0
        elif x <= (11.0 - np.sqrt(5.0)) / 2.0:
            return 3.0 - np.sqrt(5.0)
        else:
            return 2.0 * x - 8.0

    def Fphi(self, x):
        inv = self.iphi(x)
        return self.F(inv)

    def F(self, x):
        if x < 0:
            return 0
        elif x < 3:
            return 0.75 * x / 3
        elif x <= 8:
            return 0.75 + (x - 3) / 5 * 0.25
        else:

        # print(x,"invalid CDF value")

            return 1

    def iF(self, x):
        if x <= 0.75:
            return 3 * x / 0.75
        else:
            return (x - 0.75) * 20 + 3

    def iphi(self, x):
        if x < 3.0 - np.sqrt(5.0):
            return (x + 4) / 2
        elif x > 3.0 - np.sqrt(5.0):
            return (x + 8) / 2
        else:

        # print("inverse not properly defined")

            return (9.0 - np.sqrt(5.0)) / 2.0

    def iphiv(self, x):
        mask1 = x <= 3.0 - np.sqrt(5.0)
        mask2 = x > 3.0 - np.sqrt(5.0)
        output = x.copy()
        output[mask1] = (output[mask1] + 4) / 2
        output[mask2] = (output[mask2] + 8) / 2
        return output

    def alloc(self, arr):
        vstar = self.iF(self.thetamax)
        v2r = arr[:, 1]
        v1r = self.phi(arr[:, 0]) / self.alpha
        cpr = np.maximum(self.iphiv(v1r), vstar)

        return (v2r <= cpr).astype(int)

    def interimalloc(self, x):
        vstar = self.iF(self.thetamax)
        fstterm = max(self.F(vstar), self.F(self.iphi(self.scalarphi(x)
                      / self.alpha)))
        return fstterm + self.alpha * int(x > vstar) \
            * self.F(self.iphi(self.alpha * self.scalarphi(x))) \
            - self.alpha

    def interimalloc2(self, x):
        vstar = self.iF(self.thetamax)
        fstterm = max(self.Fphi(self.scalarphi(vstar)),
                      self.Fphi(self.scalarphi(x) / self.alpha))
        return fstterm + self.alpha * int(x > vstar) \
            * self.Fphi(self.alpha * self.scalarphi(x)) - self.alpha

    def scipyprice(self, x):
        return x * self.interimalloc(x) - integrate.quad(lambda x: \
                self.interimalloc(x), 0, x, limit=150)[0]

    def error(self, x):
        return integrate.quad(lambda x: self.interimalloc(x), 0, x,
                              limit=150)[1]

    def samplers(self, batch_size):
        spl1 = np.random.uniform(low=0.0, high=3.0, size=batch_size)
        spl2 = np.random.uniform(low=3.0, high=8.0, size=batch_size)
        choice = np.random.rand(batch_size)
        choice1 = choice > 0.75
        result = spl1.copy()
        result[choice1] = spl2[choice1]
        return result

    def revenue(self, batchsize):
        x = self.samplers(batchsize).reshape(-1, 2)
        y = x.copy()
        y[:, 0] = x[:, 1]
        y[:, 1] = x[:, 0]
        return ((self.phi(x[:, 0]) - self.alpha * self.phi(x[:, 1]))
                * self.alloc(x)).mean() + ((self.phi(x[:, 1])
                - self.alpha * self.phi(x[:, 0]))
                * self.alloc(y)).mean()

    def revenue2(self, batchsize):
        x = self.samplers(batchsize)
        ctr = 0
        count = 0
        for i in x:
            ctr += self.interimalloc(i) * self.scalarphi(i) * 2
            count += 1
        return ctr / count


In [8]:
alpha_values = [0.5, 2.0]
theta_values = [0.5, 0.75]
batch_size = 2**29

for alpha in alpha_values:
    for theta in theta_values:
        expo_alloc = ExpoAlloc(1, alpha, theta)
        uniform_alloc = Uniform1alloc(alpha, theta)
        asym_uniform = AsymUniform(alpha, theta)


        expo_revenue = expo_alloc.revenue(batch_size)
        uniform_revenue = uniform_alloc.revenue(batch_size)
        asym_uniform_revenue = asym_uniform.revenue(batch_size)

        print(f"Alpha: {alpha}, Theta: {theta}")
        print("ExpoAlloc Revenue:", expo_revenue)
        print("Uniform1alloc Revenue:", uniform_revenue)
        print("AsymUniform Revenue:", asym_uniform_revenue)
        print("-------------------------")

Alpha: 0.5, Theta: 0.5
ExpoAlloc Revenue: 0.6325882700074241
Uniform1alloc Revenue: 0.39577126397046125
AsymUniform Revenue: 0.6094263565366813
-------------------------
Alpha: 0.5, Theta: 0.75
ExpoAlloc Revenue: 0.44770641468882016
Uniform1alloc Revenue: 0.2370413247933133
AsymUniform Revenue: 0.36918099220458045
-------------------------
Alpha: 2.0, Theta: 0.5
ExpoAlloc Revenue: 1.6129327790569965
Uniform1alloc Revenue: 1.0416690763003005
AsymUniform Revenue: 1.5936744557689355
-------------------------
Alpha: 2.0, Theta: 0.75
ExpoAlloc Revenue: 1.41475863257538
Uniform1alloc Revenue: 0.7500088232082645
AsymUniform Revenue: 1.1353259170496355
-------------------------


In [9]:
# Irregular dist
alpha_values = [0.5, 2.0]
theta = 0.5
batch_size = 2**29

for alpha in alpha_values:
    irregular = Irregular(alpha, theta)

    irregular_revenue = irregular.revenue(batch_size)

    print(f"Alpha: {alpha}, Theta: 0.5")
    print("Irregular Revenue:", irregular_revenue)
    print("-------------------------")

Alpha: 0.5, Theta: 0.5
Irregular Revenue: 1.698354493663489
-------------------------
Alpha: 2.0, Theta: 0.5
Irregular Revenue: 4.39698709594011
-------------------------
